大语言模型 预测下一个词的概率分布 + 有监督的训练方式
1. 基础语言模型的回答是和语料有关（回答是语义相似未必是答案）
2. 指令微调的语言模型：指令调优，任务导向

LLM （large language model）预测是下一个token。简单单词多个转为1个token，相反复杂单词甚至一个单词拆成多个token


In [23]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai_api_key = os.environ['OPENAI_API_KEY']

from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)


def gen_gpt_messages(prompt):
    messages = [{"role": "user", "content": prompt}]
    return messages


def get_completion(prompt, model="gpt-3.5-turbo", temperature = 0):
    response = client.chat.completions.create(
        model=model,
        messages=gen_gpt_messages(prompt),
        temperature=temperature,
    )
    if len(response.choices) > 0:
        return response.choices[0].message.content
    return "generate answer error"


#### 提问范式和token

In [ ]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature = 0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    if len(response.choices) > 0:
        return response.choices[0].message.content
    return "generate answer error"

def get_completion_and_token_count(messages, 
                                   model="gpt-3.5-turbo", 
                                   temperature=0, 
                                   max_tokens=500):
    """
    使用 OpenAI 的 GPT-3 模型生成聊天回复，并返回生成的回复内容以及使用的 token 数量。

    参数:
    messages: 聊天消息列表。
    model: 使用的模型名称。默认为"gpt-3.5-turbo"。
    temperature: 控制生成回复的随机性。值越大，生成的回复越随机。默认为 0。
    max_tokens: 生成回复的最大 token 数量。默认为 500。

    返回:
    content: 生成的回复内容。
    token_dict: 包含'prompt_tokens'、'completion_tokens'和'total_tokens'的字典，分别表示提示的 token 数量、生成的回复的 token 数量和总的 token 数量。
    """
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    if len(response.choices) > 0:
        result = response.choices[0].message.content
        token_dict = {
            'prompt_tokens':response.usage.prompt_tokens,
            'completion_tokens':response.usage.completion_tokens,
            'total_tokens':response.usage.total_tokens,
        }
        return result, token_dict
    return "generate answer error"

不常用的词，颠倒这个词会出现错误。如果加上分词器，分解成3个token，出现问题

In [3]:
response = get_completion("Take the letters in lollipop \
and reverse them")
print(response)

pilpolol


In [4]:
response = get_completion("""Take the letters in \
l-o-l-l-i-p-o-p and reverse them""")

print(response)

p-o-p-i-l-l-o-l


英文输入：token为4个字符，或者3/4个单词

中文输入：token为一个或半个词

token为：输入和completion的输出的总和，输入越多，输出字数也就越少，比如turbo-3.5为4096


+ 系统消息：向语言模型传达讯息
+ 用户消息：模拟用户的问题

In [12]:
messages = [
    {'role':'system',
     'content':'你是一个助理，并以 Seuss 苏斯博士的风格作出回答。'},
     {'role':'user',
      'content':'就快乐的小鲸鱼为主题给我写一首短诗'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

在大海里游游戏，
欢快的小鲸鱼唱歌。
尾巴翘翘摇来摇去，
快乐永不停歇。

海底花园美如画，
小鲸鱼跳跃欢天涯。
蓝色水波荡漾心，
快乐在每个角落寻。


In [13]:
# 长度控制
messages =  [  
{'role':'system',
 'content':'你的所有答复只能是一句话'},    
{'role':'user',
 'content':'写一个关于快乐的小鲸鱼的故事'},  
] 
response = get_completion_from_messages(messages, temperature=1)
print(response)

一只快乐的小鲸鱼在蔚蓝的海洋中跳跃游戏，和海底的朋友们一起分享欢笑和快乐的时光。


In [14]:
# 以上结合
messages =  [  
{'role':'system',
 'content':'你是一个助理， 并以 Seuss 苏斯博士的风格作出回答，只回答一句话'},    
{'role':'user',
 'content':'写一个关于快乐的小鲸鱼的故事'},
] 
response = get_completion_from_messages(messages, temperature =1)
print(response)

在大海深处有只小鲸鱼，欢快地游来游去，尾巴摆动得欢愉，快乐永远伴随着它。


In [21]:
messages =  [  
{'role':'system', 
 'content':'你是一个助理， 并以 Seuss 苏斯博士的风格作出回答。'},    
{'role':'user', 
 'content':'就快乐的小鲸鱼为主题给我写一首短诗'},  
] 
response, token_dict = get_completion_and_token_count(messages)
print(response)

在大海深处有一只小鲸鱼，
它快乐地游来游去，不停歇。
尾巴摆动，水花飞溅，
欢乐的歌声响彻海洋。

它和海豚一起跳舞，
在阳光下闪闪发亮。
快乐的小鲸鱼，永不疲倦，
让每个人都感到温暖和欢欣。


In [22]:
print(token_dict)

{'prompt_tokens': 67, 'completion_tokens': 132, 'total_tokens': 199}


#### 分类

对用户查询的指令进行分类，分隔符区分指令和输出，一般用”#“，被模型认为是一个单独token

In [24]:
delimiter = "####"

system_message=f"""
你将获得客户服务查询。
每个客户服务查询都将用{delimiter}字符分隔。
将每个查询分类到一个主要类别和一个次要类别中。
以JSON格式提供你的输出，包含以下键：primary 和 secondary。

主要类别：计费（Billing）、技术支持（Technical Support）、账户管理（Account Management）或一般咨询（General Inquiry）。

计费次要类别：
取消订阅或升级（Unsubscribe or upgrade）
添加付款方式（Add a payment method）
收费解释（Explanation for charge）
争议费用（Dispute a charge）

技术支持次要类别：
常规故障排除（General troubleshooting）
设备兼容性（Device compatibility）
软件更新（Software updates）

账户管理次要类别：
重置密码（Password reset）
更新个人信息（Update personal information）
关闭账户（Close account）
账户安全（Account security）

一般咨询次要类别：
产品信息（Product information）
定价（Pricing）
反馈（Feedback）
与人工对话（Speak to a human）
"""

user_message=f"""\
我希望你删除我所有的个人资料和所有用户数据。"""

messages = [
{'role':'system',
 'content':system_message},
 {'role':'user',
 'content':f"{delimiter}{user_message}{delimiter}"}
]

In [30]:
from chat_tools import get_completion_from_messages

response = get_completion_from_messages(messages)
print(response)

{
    "primary": "账户管理",
    "secondary": "关闭账户"
}


In [32]:
user_message = f"""\
请告诉我更多有关你们的平板电脑的信息"""
messages = [
    {'role':'system',
     'content':system_message},
     {'role':'user',
      'content':user_message}
]
response = get_completion_from_messages(messages)
print(response)

{
    "primary": "General Inquiry",
    "secondary": "Product information"
}####
